# L=2 BMN SU2 matrix model

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import qiskit
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit_algorithms import NumPyEigensolver
import time
import sys
sys.path.append('../utility')
from vqe_run import *
from qc_ansatze import *

# Load H

In [2]:
def open_n_process(file_link):
    file = open(file_link, "r")
    content = file.read()

    data = content.split(',\n')
    data[0] = data[0].split('\n')[1]
    data[-1] = data[-1].split('\n')[0]

    data_coeff = []
    data_str = []
    for i in range(len(data)):
        data_coeff.append(float(data[i].split('*')[0]))
        data_str.append(data[i].split('*')[1].split(' ')[1])
    return data_coeff, data_str
    
def check_min_max(data_coeff):
    print(f'Min absolute value is {np.round(np.min(np.abs(data_coeff[1:])),6)}')
    print(f'Max absolute value is {np.round(np.max(np.abs(data_coeff[1:])),5)}')
    print(f'Mean absolute value is {np.round(np.mean(np.abs(data_coeff[1:])),5)}')
    
def get_k_largest_ops(data_coeff, data_str, k):
    ind_list = np.argsort(np.abs(data_coeff[1:]))[-k:]
    ops = [data_str[1:][i] for i in ind_list]
    val= [np.round(data_coeff[1:][i],4) for i in ind_list]
    return ind_list, ops, val

In [3]:
gg = '10'
data_coeff, data_str = open_n_process(f"../utility/L2_BMN_l{gg}_Hpauli.txt")

In [4]:
Hpauli = list(zip(data_str, data_coeff))
H4q = SparsePauliOp.from_list(Hpauli)
# exactly diagonalize the system using numpy routines
solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
#print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],6)
print(f'{check_min_max(data_coeff)}')
print(f'E_exact = {E_exact}')

Min absolute value is 0.25
Max absolute value is 0.75
Mean absolute value is 0.42678
None
E_exact = 0.04829


In [5]:
Hpauli

[('IIIIIIIII', 6.0),
 ('ZIIIIIIII', -0.5),
 ('IZIIIIIII', -0.5),
 ('IIZIIIIII', -0.5),
 ('IIIZIIIII', -0.5),
 ('IIIIZIIII', -0.5),
 ('IIIIIZIII', -0.5),
 ('IIIIIIZII', -0.75),
 ('IIIIIIIZI', -0.75),
 ('IIIIIIIIZ', -0.75),
 ('XXIXXIIII', -0.24999999999999992),
 ('XIXXIXIII', -0.24999999999999992),
 ('IXXIXXIII', -0.24999999999999992),
 ('IIXIIIYXI', 0.35355339059327373),
 ('IIXIIIXYI', 0.35355339059327373),
 ('IIIIIXXXI', 0.35355339059327373),
 ('IIIIIXYYI', -0.35355339059327373),
 ('IXIIIIYZX', -0.35355339059327373),
 ('IXIIIIXZY', -0.35355339059327373),
 ('IIIIXIXZX', -0.35355339059327373),
 ('IIIIXIYZY', 0.35355339059327373),
 ('XIIIIIIYX', 0.35355339059327373),
 ('XIIIIIIXY', 0.35355339059327373),
 ('IIIXIIIXX', 0.35355339059327373),
 ('IIIXIIIYY', -0.35355339059327373)]

# EvolvedOperatorAnsatz

In [5]:
ind15, ops15, vals15 = get_k_largest_ops(data_coeff, data_str, 15)
ind20, ops20, vals20 = get_k_largest_ops(data_coeff, data_str, 20)

ops_Hp15 =[Pauli(f'{ops15[i]}') for i in range(len(ops15))]
ops_Hp20 =[Pauli(f'{ops20[i]}') for i in range(len(ops20))]
ops_H =[Pauli(f'{data_str[1:][i]}') for i in range(len(data_str[1:]))]

In [6]:
ev_op_Hp15 = EvolvedOperatorAnsatz(ops_Hp15, reps=1, insert_barriers=True)
ev_op_Hp20 = EvolvedOperatorAnsatz(ops_Hp20, reps=1, insert_barriers=True)
ev_op_H = EvolvedOperatorAnsatz(ops_H, reps=1, insert_barriers=True)

ev_op_Hp15_2f = EvolvedOperatorAnsatz(ops_Hp15, reps=2, insert_barriers=True)
ev_op_Hp20_2f = EvolvedOperatorAnsatz(ops_Hp20, reps=2, insert_barriers=True)
ev_op_H_2f = EvolvedOperatorAnsatz(ops_H, reps=2, insert_barriers=True)

ev_op_Hp15_3f = EvolvedOperatorAnsatz(ops_Hp15, reps=3, insert_barriers=True)
ev_op_Hp20_3f = EvolvedOperatorAnsatz(ops_Hp20, reps=3, insert_barriers=True)
ev_op_H_3f = EvolvedOperatorAnsatz(ops_H, reps=3, insert_barriers=True)

ev_op_Hp15_4f = EvolvedOperatorAnsatz(ops_Hp15, reps=4, insert_barriers=True)
ev_op_Hp20_4f = EvolvedOperatorAnsatz(ops_Hp20, reps=4, insert_barriers=True)
ev_op_H_4f = EvolvedOperatorAnsatz(ops_H, reps=4, insert_barriers=True)

ansatz_list = [ev_op_Hp15, ev_op_Hp20, ev_op_H, 
               ev_op_Hp15_2f, ev_op_Hp20_2f, ev_op_H_2f,
               ev_op_Hp15_3f, ev_op_Hp20_3f, ev_op_H_3f,
               ev_op_Hp15_4f, ev_op_Hp20_4f, ev_op_H_4f]

ansatz_names = ['ev_op_Hp15', 'ev_op_Hp20', 'ev_op_H', 
               'ev_op_Hp15_2f','ev_op_Hp20_2f','ev_op_H_2f',
               'ev_op_Hp15_3f','ev_op_Hp20_3f','ev_op_H_3f',
               'ev_op_Hp15_4f','ev_op_Hp20_4f','ev_op_H_4f']

print(f'number of params: {[ansatz_list[i].num_parameters for i in range(len(ansatz_list))]}')

number of params: [15, 20, 24, 30, 40, 48, 45, 60, 72, 60, 80, 96]


In [8]:
print(*list(zip(ops20, vals20)), sep = '\n')

('XIIIIIIYX', 0.3536)
('IIIIXIYZY', 0.3536)
('IIIIXIXZX', -0.3536)
('IXIIIIXZY', -0.3536)
('IXIIIIYZX', -0.3536)
('IIIIIXYYI', -0.3536)
('IIIIIXXXI', 0.3536)
('IIXIIIXYI', 0.3536)
('IIIXIIIYY', -0.3536)
('IIIXIIIXX', 0.3536)
('IIXIIIYXI', 0.3536)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


In [9]:
print(*list(zip(ops15, vals15)), sep = '\n')

('IIIIIXYYI', -0.3536)
('IIIIIXXXI', 0.3536)
('IIXIIIXYI', 0.3536)
('IIIXIIIYY', -0.3536)
('IIIXIIIXX', 0.3536)
('IIXIIIYXI', 0.3536)
('IIIIIZIII', -0.5)
('IIIIZIIII', -0.5)
('IIIZIIIII', -0.5)
('IIZIIIIII', -0.5)
('IZIIIIIII', -0.5)
('ZIIIIIIII', -0.5)
('IIIIIIIIZ', -0.75)
('IIIIIIIZI', -0.75)
('IIIIIIZII', -0.75)


# VQE

In [7]:
seed = 170
iterations = 2000
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(iterations, optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.6f}")
    return result

# COBYLA

At step 6, with ev_op_Hp15_3f
VQE result: 0.397219
Length of this optimization 531, time taken = 73.975 

In [11]:
r_cobyla=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, COBYLA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_cobyla.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))
    

At step 0, with ev_op_Hp15
VQE result: 2.345350
Length of this optimization 200, time taken = 22.056 

At step 1, with ev_op_Hp20
VQE result: 0.462793
Length of this optimization 295, time taken = 54.336 

At step 2, with ev_op_H
VQE result: 2.690010
Length of this optimization 319, time taken = 49.951 

At step 3, with ev_op_Hp15_2f
VQE result: 0.554531
Length of this optimization 351, time taken = 42.255 

At step 4, with ev_op_Hp20_2f
VQE result: 0.408860
Length of this optimization 522, time taken = 102.828 

At step 5, with ev_op_H_2f
VQE result: 0.648100
Length of this optimization 805, time taken = 192.001 

At step 6, with ev_op_Hp15_3f
VQE result: 0.397219
Length of this optimization 531, time taken = 84.777 

At step 7, with ev_op_Hp20_3f
VQE result: 2.340578
Length of this optimization 972, time taken = 264.111 

At step 8, with ev_op_H_3f
VQE result: 3.733768
Length of this optimization 899, time taken = 305.177 

At step 9, with ev_op_Hp15_4f
VQE result: 0.576417
Length of

In [12]:
df1 = pd.concat([r_cobyla[i] for i in range(len(r_cobyla))], axis = 1)
df1.to_csv(f'L2_l{gg}_op_ev_cobyla.csv')

# SPSA

At step 1, with ev_op_Hp20
VQE result: 0.124915
Length of this optimization 3551, time taken = 348.957

At step 1, with ev_op_Hp20
VQE result: 0.164071
Length of this optimization 4051, time taken = 415.244 

In [13]:
r_spsa=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SPSA, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_spsa.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 0.472914
Length of this optimization 4051, time taken = 262.144 

At step 1, with ev_op_Hp20
VQE result: 0.164071
Length of this optimization 4051, time taken = 415.244 

At step 2, with ev_op_H
VQE result: 0.678066
Length of this optimization 4051, time taken = 465.439 

At step 3, with ev_op_Hp15_2f
VQE result: 0.444836
Length of this optimization 4051, time taken = 363.858 

At step 4, with ev_op_Hp20_2f
VQE result: 0.132800
Length of this optimization 4051, time taken = 601.959 

At step 5, with ev_op_H_2f
VQE result: 1.291555
Length of this optimization 4051, time taken = 708.219 

At step 6, with ev_op_Hp15_3f
VQE result: 0.500908
Length of this optimization 4051, time taken = 470.007 

At step 7, with ev_op_Hp20_3f
VQE result: 0.443965
Length of this optimization 4051, time taken = 780.585 

At step 8, with ev_op_H_3f
VQE result: 2.003623
Length of this optimization 4051, time taken = 923.514 

At step 9, with ev_op_Hp15_4f
VQE result: 0.55

In [14]:
df2 = pd.concat([r_spsa[i] for i in range(len(r_spsa))], axis = 1)
df2.to_csv(f'L2_l{gg}_op_ev_spsa.csv')

## Retrain best ansatz

At step 4, with ev_op_Hp20_2f
VQE result: 0.132800
Length of this optimization 4051, time taken = 601.959 

At step 1, with ev_op_Hp20
VQE result: 0.124915
Length of this optimization 3551, time taken = 348.957


In [10]:
#Increasing the number of iters worsens the result
iterations = 2500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20_2f)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 0.207323
Length of this optimization 5051, time taken = 957.077 



In [11]:
iterations = 1750
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20_2f)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 0.171218
Length of this optimization 3551, time taken = 632.164 



In [9]:
iterations = 1750
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20_2f)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_b = counts
values_b = values

VQE result: 0.186749
Length of this optimization 3551, time taken = 335.473 



In [12]:
iterations = 1500
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 2.122449
Length of this optimization 3051, time taken = 400.651 



In [8]:
iterations = 1750
counts = []
values = []
t0 = time.time()
result = run_qve_w_specified_optimizer(iterations, SPSA, ev_op_Hp20)
t1 = time.time()
print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
counts_a = counts
values_a = values

VQE result: 0.132078
Length of this optimization 3551, time taken = 339.94 



In [9]:
df = pd.DataFrame({'ev_op_Hp20': values_a})
df.to_csv(f'results/L2_l{gg}_op_ev_spsa_best.csv')
df

,ev_op_Hp20
0,5.744443
1,6.533343
2,5.869314
3,6.252109
4,5.631245
...,...
3546,0.252431
3547,0.184026
3548,0.779484
3549,0.699598


# SLSQP

In [15]:
r_slsqp=[]
for i in range(len(ansatz_list)):
    print(f'At step {i}, with {ansatz_names[i]}')
    counts = []
    values = []
    t0 = time.time()
    result = run_qve_w_specified_optimizer(iterations, SLSQP, ansatz_list[i])
    t1 = time.time()
    print(f'Length of this optimization {len(values)}, time taken = {np.round(t1-t0,3)} \n')
    counts_a = counts
    values_a = values 
    r_slsqp.append(pd.DataFrame({f'{ansatz_names[i]}': values_a}))

At step 0, with ev_op_Hp15
VQE result: 6.427230
Length of this optimization 16, time taken = 2.707 

At step 1, with ev_op_Hp20
VQE result: 5.730374
Length of this optimization 21, time taken = 3.672 

At step 2, with ev_op_H
VQE result: 6.422230
Length of this optimization 25, time taken = 4.368 

At step 3, with ev_op_Hp15_2f
VQE result: 5.111638
Length of this optimization 31, time taken = 4.023 

At step 4, with ev_op_Hp20_2f
VQE result: 5.552253
Length of this optimization 41, time taken = 8.986 

At step 5, with ev_op_H_2f
VQE result: 5.934838
Length of this optimization 49, time taken = 17.323 

At step 6, with ev_op_Hp15_3f
VQE result: 4.515321
Length of this optimization 46, time taken = 10.181 

At step 7, with ev_op_Hp20_3f
VQE result: 6.339231
Length of this optimization 61, time taken = 17.801 

At step 8, with ev_op_H_3f
VQE result: 5.823893
Length of this optimization 73, time taken = 25.654 

At step 9, with ev_op_Hp15_4f
VQE result: 4.803526
Length of this optimization

In [16]:
df3 = pd.concat([r_slsqp[i] for i in range(len(r_slsqp))], axis = 1)
df3.to_csv(f'L2_l{gg}_op_ev_slsqp.csv')